import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

import findspark
findspark.init()

In [2]:
#Check that installation is succesful 
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))


JAVA_HOME: /usr/lib/jvm/java-8-openjdk-amd64
SPARK_HOME: /opt/spark


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank
from pyspark.sql.functions import when

In [5]:
%%time
#write it to measure time

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 3.1 μs


create local SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MyNotebookApp") \
    .getOrCreate()

25/06/20 11:33:43 WARN Utils: Your hostname, athena resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/20 11:33:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/06/20 11:33:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


read csv with inferschema

In [5]:
%%time
df1 = spark.read.option("header", True).option("inferSchema", True).csv("ds_salaries.csv")

CPU times: user 0 ns, sys: 4.52 ms, total: 4.52 ms
Wall time: 2.6 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [9]:
%%time
df1 = spark.read.option("header", True).option("inferSchema", True).csv("ds_salaries.csv")

CPU times: user 3.13 ms, sys: 0 ns, total: 3.13 ms
Wall time: 145 ms


write schema of scv on screen

In [10]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [5]:
schema = StructType() \
    .add("_c0", IntegerType(), True) \
    .add("work_year", IntegerType(), True) \
    .add("experience_level", StringType(), True) \
    .add("employment_type", StringType(), True) \
    .add("job_title", StringType(), True) \
    .add("salary", IntegerType(), True) \
    .add("salary_currency", StringType(), True) \
    .add("salary_in_usd", IntegerType(), True) \
    .add("employee_residence", StringType(), True) \
    .add("remote_ratio", IntegerType(), True) \
    .add("company_location", StringType(), True) \
    .add("company_size", StringType(), True) 



restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [6]:
%%time
df_with_schema = spark.read.format("csv").option("header", True) \
    .schema(schema) \
    .load("ds_salaries.csv")

CPU times: user 4.46 ms, sys: 0 ns, total: 4.46 ms
Wall time: 966 ms


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [13]:
df_with_schema.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: integer (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [7]:
df_with_schema.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

25/06/20 11:34:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/hello/projects/spark_demo_course_task/1_PySpark_Basics/ds_salaries.csv


print data in dataframe using display(df.toPandas())

In [9]:
from IPython.display import display
display(df_with_schema.limit(5).toPandas())

25/06/20 11:35:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/hello/projects/spark_demo_course_task/1_PySpark_Basics/ds_salaries.csv


,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


create df_job_title that consists from all job_titles without duplicates

In [10]:
df = df_with_schema
df_job_title = df.dropDuplicates(["job_title"]).select("job_title")
df_job_title.show()

+--------------------+
|           job_title|
+--------------------+
|3D Computer Visio...|
|  Lead Data Engineer|
|Head of Machine L...|
|     Data Specialist|
| Data Analytics Lead|
|Machine Learning ...|
|   Lead Data Analyst|
|Data Engineering ...|
|Staff Data Scientist|
|       ETL Developer|
|Director of Data ...|
|Product Data Analyst|
|Principal Data Sc...|
|        AI Scientist|
|Director of Data ...|
|Machine Learning ...|
| Lead Data Scientist|
|Machine Learning ...|
|Data Science Engi...|
|Machine Learning ...|
+--------------------+
only showing top 20 rows



print all rows from df_job_titles without truncating jobs

In [11]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [12]:
df_analytic = df.groupBy("job_title") \
    .agg(
        max("salary_in_usd").alias("max_salary"),
        avg("salary_in_usd").alias("avg_salary"),
        min("salary_in_usd").alias("min_salary"),
    )

print all rows from df_analytic without trancating jobs

In [14]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
from pyspark.sql.functions import desc
windowSpec = Window.orderBy(desc("avg_salary"))

df_analytic = df_analytic.withColumn("row_id", row_number().over(windowSpec))

print all data from df_analytic

In [18]:
df_analytic.show(truncate=False)

25/06/20 11:42:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------------------+----------+------------------+----------+------+
|job_title                         |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------+----------+------------------+----------+------+
|Data Analytics Lead               |405000    |405000.0          |405000    |1     |
|Principal Data Engineer           |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst            |450000    |275000.0          |100000    |3     |
|Principal Data Scientist          |416000    |215242.42857142858|148261    |4     |
|Director of Data Science          |325000    |195074.0          |130026    |5     |
|Data Architect                    |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist            |380000    |175655.0          |54238     |7     |
|Analytics Engineer                |205300    |175000.0          |135000    |8     |
|Data Specialist                   |165000    |165000.0          

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [19]:
new_cols = [df_analytic.columns[-1]] + df_analytic.columns[:-1]
df_analytic = df_analytic.select(new_cols)

print df_analytic now

In [20]:
df_analytic.show(truncate=False)

25/06/20 11:44:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+----------------------------------+----------+------------------+----------+
|row_id|job_title                         |max_salary|avg_salary        |min_salary|
+------+----------------------------------+----------+------------------+----------+
|1     |Data Analytics Lead               |405000    |405000.0          |405000    |
|2     |Principal Data Engineer           |600000    |328333.3333333333 |185000    |
|3     |Financial Data Analyst            |450000    |275000.0          |100000    |
|4     |Principal Data Scientist          |416000    |215242.42857142858|148261    |
|5     |Director of Data Science          |325000    |195074.0          |130026    |
|6     |Data Architect                    |266400    |177873.9090909091 |90700     |
|7     |Applied Data Scientist            |380000    |175655.0          |54238     |
|8     |Analytics Engineer                |205300    |175000.0          |135000    |
|9     |Data Specialist                   |165000    |165000.0   

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

тут я сделал обычный груп бай и оставил только две колонки. В описании сказано "сохранить все колонки", тогда нужно делать оконной функцией, а в таком случае ломается смысл следующего таска (df_best) тк у нас итак будет нужная таблица (за исключением того что там нужно будет выбрать только 4 колонки). Поэтому тут я остановился на груп бай, чтобы красиво связать со следующим таском. А дальше там уже будет пример как я исползую оконку

In [21]:
df_exp_lvl = df.groupBy("experience_level").max("salary_in_usd")

print here df_exp_lvl

In [22]:
df_exp_lvl.show()

+----------------+------------------+
|experience_level|max(salary_in_usd)|
+----------------+------------------+
|              EX|            600000|
|              MI|            450000|
|              EN|            250000|
|              SE|            412000|
+----------------+------------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [23]:
from pyspark.sql.functions import broadcast
df_exp_lvl = df_exp_lvl.withColumnRenamed("max(salary_in_usd)", "biggest_salary")

df_joined_exp = df.join(broadcast(df_exp_lvl), on = "experience_level").withColumnRenamed("_c0", "id")  

df_best = df_joined_exp.filter(df_joined_exp["salary_in_usd"] == df_joined_exp["biggest_salary"]).select("id", "experience_level", "biggest_salary", "employee_residence")


print df_best

In [24]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



25/06/20 12:04:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/hello/projects/spark_demo_course_task/1_PySpark_Basics/ds_salaries.csv


drop duplicates if exist by experience_level

In [25]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [26]:
df_best.show()

25/06/20 12:05:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , experience_level, salary_in_usd, employee_residence
 Schema: _c0, experience_level, salary_in_usd, employee_residence
Expected: _c0 but found: 
CSV file: file:///home/hello/projects/spark_demo_course_task/1_PySpark_Basics/ds_salaries.csv


+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [31]:
# from pyspark.sql.functions import udf

# def new_exp_lvl(s):
#     if s == "MI":
#         return "middle"
#     if s == "SE":
#         return "senior"
#     return None

# new_exp_lvl_udf = udf(new_exp_lvl, StringType())

from pyspark.sql.functions import lit

df_new_best = df_best.withColumn("experience_level", when(col("experience_level") == "MI", "middle")
                                 .when(col("experience_level") == "SE", "senior")
                                 .otherwise(lit(None))
        ).drop("id")

print df_new_best

In [32]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        600000|                US|
|          middle|        450000|                US|
|            null|        250000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [41]:
df_new_best.coalesce(1).write.option("header", True).csv("1.csv")



In [42]:
df_final = spark.read.option("header", True).option("inferSchema", True).csv("1.csv")

print df_final

In [43]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        600000|                US|
|          middle|        450000|                US|
|            null|        250000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [46]:
df_final_1 = df_final.filter(col("experience_level").isNotNull())

In [47]:
df_final_1.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



In [48]:
df_final_joined = df.join(broadcast(df_final_1), (df["salary_in_usd"] == df_final_1["biggest_salary"]) & (df["employee_residence"] == df_final_1["employee_residence"]))

print df_final

In [49]:
df_final_joined.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+------------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|experience_level|biggest_salary|employee_residence|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+------------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|          middle|        450000|                US|
| 63|     2020|              SE|             FT|      Data Scientist|412000|            USD|       412000|              

25/06/20 12:25:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/hello/projects/spark_demo_course_task/1_PySpark_Basics/ds_salaries.csv


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [57]:
max_salary = df_final_joined.groupBy().max("biggest_salary").first()[0]
print(max_salary)


450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system